# Import

# Find most similar

In [21]:
with open('data/corona_info.txt', 'r', encoding="utf8") as f:
    blob = f.read().replace("\n", " ")

In [75]:
import numpy as np
import gensim
from scipy import spatial
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import sent_tokenize
import numpy as np
from nltk.corpus import stopwords 

class TextSimilarity:

    def __init__(self):
        try:
            self.model = gensim.models.KeyedVectors.load_word2vec_format('wiki-news-300d-1M.vec')
            self.index2word_set = set(self.model.wv.index2word)
        except FileNotFoundError:
            raise FileNotFoundError
            
    def avg_feature_vector(self, sentence, num_features=300):
        words = WordPunctTokenizer().tokenize(sentence)
        feature_vec = np.zeros((num_features, ), dtype='float32')
        n_words = 0
        for word in words:
            if word in self.index2word_set:
                n_words += 1
                feature_vec = np.add(feature_vec, self.model[word])
#         if (n_words > 0):
#             feature_vec = np.divide(feature_vec, n_words)
        return feature_vec

    def similarity(self, sentence1, sentence2):
        vec1, vec2 = self.avg_feature_vector(sentence1), self.avg_feature_vector(sentence2)
        return self.cosine_similarity(vec1, vec2)

    def cosine_similarity(self, vec1, vec2):
        return 1 - spatial.distance.cosine(vec1, vec2)

sim = TextSimilarity()

def find_similar(sent, blob):
    sentences = np.asarray(sent_tokenize(blob))
    sent_blob_sim = np.asarray([sim.similarity(sent, sentences[i]) for i in range(len(sentences))])
    indices_sorted = np.argsort(sent_blob_sim)[::-1]
    sent_sim_sorted = sentences[indices_sorted]
    return sent_sim_sorted[0]

<ipython-input-75-4c56361686a3>:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  self.index2word_set = set(self.model.wv.index2word)


In [142]:
with open('data/corona_info.txt', 'r', encoding="utf8") as f:
    blob = f.read().replace("\n", " ")

with open('data/corona_questions.txt', 'r', encoding="utf8") as f:
    claims = f.read().split("\n")

sw = stopwords.words('english') 
# skip_word = ['corona', 'virus', 'covid-19', 'coronavirus']
for claim in claims:
#     claim_filtered = ' '.join([x for x in claim.lower().split() if x not in skip_word])
    print("Factual Claim:", claim, '\n')
    print("Check:", '\n' + find_similar(claim, blob))
    print("----------\n\n")

Factual Claim: I should not contact the hospital if i felt any symptoms  

Check: 
If your symptoms are severe or you feel like you need medical care, call before you go to a doctor’s office, urgent care center or emergency room.
----------


Factual Claim: Fever is the only COVID-19 symptom 

Check: 
COVID-19 symptoms include: Cough Fever Shortness of breath Muscle aches Sore throat Unexplained loss of taste or smell Diarrhea Headache   In rare cases, COVID-19 can lead to severe respiratory problems, kidney failure or death.
----------


Factual Claim: diagnosing corona virus is very easy 

Check: 
The virus that causes COVID-19 is similar to the one that caused the 2003 SARS outbreak: both are types of coronaviruses.
----------


Factual Claim: there currently exist a cure or a medication to treat corona virus 

Check: 
Currently, no antiviral medication is recommended to treat COVID-19 , and no cure is available for COVID-19, Antibiotics aren't effective against viral infections.
--

In [34]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')

X ="I love horror movies".lower()
Y ="Lights out is love a horror movies".lower()

def similar_scorer(X, Y):
    # tokenization 
    X_list = word_tokenize(X) 
    Y_list = word_tokenize(Y) 

    # sw contains the list of stopwords 
    sw = stopwords.words('english') 
    l1 =[];l2 =[] 

    # remove stop words from string 
    X_set = {w for w in X_list if not w in sw} 
    Y_set = {w for w in Y_list if not w in sw} 

    # form a set containing keywords of both strings 
    rvector = X_set.union(Y_set) 
    for w in rvector: 
        if w in X_set: l1.append(1) # create a vector 
        else: l1.append(0) 
        if w in Y_set: l2.append(1) 
        else: l2.append(0) 
    c = 0

    # cosine formula 
    for i in range(len(rvector)): 
        c+= l1[i]*l2[i] 
    cosine = c / float((sum(l1)*sum(l2))**0.5) 
    return cosine

In [42]:
similar_scorer(X, Y)

0.8660254037844387